# Listing Evaluation Metrics for Each Trained Model

The aim of this file is to evaluate each of the trained files and to create a dictionary where the name of the file/folder refers to its evaluated metrics.

In [1]:
#Basics
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

#Evaluation
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, recall_score, f1_score, precision_score

#PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import SubsetRandomSampler

#Torchvision
import torchvision
from torchvision import transforms, datasets

#Data
from FRDEEP import FRDEEPF
from MiraBest import MiraBest_full

#My Functions / Models
from models_new import *
from utils import *

#Other
import pickle
from tqdm import tqdm
import PIL
from torchsummary import summary
from models.networks_other import init_weights

In [2]:
# Define transformations required for analysis (ie. func evaluate(f))
out_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize([0.5],[0.5])
                                    ])
test_transform = transforms.Compose([transforms.RandomRotation([0,360],resample=PIL.Image.BILINEAR),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5],[0.5])
                                     ])

In [3]:
# We extract two lists containing all folders and files respectively in TrainedNetworks
dirs = os.listdir('TrainedNetworks/')
folders = []
files = []
for k in dirs:
    if k[-3:]=='.pt':
        files.append(k)
    elif k[-3:]!='.pt' and k[0]!='.':
        folders.append(k)
    else:
        pass
print(f'Folders: {len(folders)}\nFiles: {len(files)}')

Folders: 181
Files: 43


In [4]:
# Evaluate all of the missing models:
def eval_models(model_paths,dataset='automatic', dates = False):
    exceptions = ['temp.pt', '0605A-MiraBestNOHYBRIDSGD-playgroundv3-0.001', '']
    
    # Either load existing dictionary or initiate an empty dictionary:
    if os.path.isfile(f'evaluated_models_{dataset}.pickle'):
        pickle_in = open(f'evaluated_models_{dataset}.pickle','rb')
        evaluated_models = pickle.load(pickle_in)
        
        pickle_in.close()
        print(f'{dataset} Successfully loaded in dict of evaluated models.')
    else:
        evaluated_models = {}
        print(f'{dataset} Initialised dict for model evaluation.')

    for idx, i in enumerate(model_paths):
        print(f'{idx}\t{i}')
        if dates:
            if (evaluated_models.get(i,False) == False) & (i not in exceptions) & (i[:4] in dates): ### Clean this so that it works on all again (or at least most with <5 exceptions).
                print(f'\tEvaluating ...')
                evaluated_models[i]=evaluate(i, dataset=dataset)
        else:
            if (evaluated_models.get(i,False) == False) & (i not in exceptions):
                print(f'\tEvaluating ...')
                evaluated_models[i]=evaluate(i, dataset=dataset)
    
    pickle_out = open(f'evaluated_models_{dataset}.pickle','wb')
    pickle.dump(evaluated_models,pickle_out)
    pickle_out.close()
    return evaluated_models

In [6]:
data_options = ['FRDEEP-F','MiraBest','MiraBestNOHYBRID','MiraBestNOUNC','automatic']
for dataset in data_options:
    print(f'--> EVALUATING ON: {dataset}')
    temp_dict = eval_models(files+folders, dataset, dates = ['0624', '0622', '0709'])

"""
dataset_selection = 'automatic'
pickle_in = open(f'evaluated_models_{dataset_selection}.pickle','rb')
evaluated_models = pickle.load(pickle_in)
pickle_in.close()
#"""

--> EVALUATING ON: FRDEEP-F
FRDEEP-F Successfully loaded in dict of evaluated models.
0	playgroundv1-0214-0.01_360Epochs.pt
1	0616temp-MiraBestNOHYBRIDAdam-AGSononet-0.001.pt
2	playgroundv1-0217-1e-07_124Epochs.pt
3	playground-0213-0.1_180Epochs.pt
4	playground-0211-0.1_360Epochs.pt
5	playground-0212-0.1_360Epochs.pt
6	playground-0207-0.1_360Epochs.pt
7	playgroundv1-0215-0.0001_124Epochs.pt
8	0702B-std_mean_norm-MiraBestNOHYBRIDAdam-AGRadGalNetdeep_sup-0.0001.pt
9	playground-0128-0.1_500Epochs.pt
10	playgroundv1-0217-0.001_124Epochs.pt
11	playground-0213-0.1_64Epochs.pt
12	playground-0129-0.1_500Epochs.pt
13	playgroundv1-0214-0.1_360Epochs.pt
14	playgroundv1-0214-0.001_361Epochs.pt
15	playground-0129-0.1_360Epochs.pt
16	playground-TESTING-0.001_124Epochs.pt
17	playgroundv1-0215-0.001_360Epochs.pt
18	AGSononet-0219-0.001_360Epochs.pt
19	playgroundv1-0213-0.1_5Epochs.pt
20	playgroundv1-0217Adam-0.0001_124Epochs.pt
21	playgroundv1-0217Adam-1e-05_360Epochs.pt
22	playgroundv1-0215-0.0001_36

/usr/local/anaconda-python-3.6/lib/python3.6/site-packages/torch/nn/functional.py:2479: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


54	0719Exp30-range_norm-MiraBestNOHYBRIDAdam-AGRadGalNetft-5e-05
55	0624Exp2-std_mean_norm-MiraBestNOHYBRIDAdam-AGRadGalNetft-0.0001
56	0611-MiraBestNOHYBRIDAdam-playgroundv3_ft-1e-06
57	0630B-std_mean_norm-MiraBestNOHYBRIDAdam-AGRadGalNetdeep_sup-0.0001
58	0305-MiraBest-playgroundv2_deep_sup
59	0429-FRDEEPAdagrad-playgroundv2_ft-0.001
60	0429-MiraBestNOUNCAdam-playgroundv2_ft-0.0001
61	0429-FRDEEPSGD-playgroundv2_ft-0.0001
62	0608-MiraBestNOHYBRIDAdadelta-playgroundv3_ft-0.01
63	0429-MiraBestNOUNCAdagrad-playgroundv2_ft-0.001
64	0611C-MiraBestNOHYBRIDSGD-playgroundv3_ft-0.0001
65	0708TEMP-range_norm-MiraBestNOHYBRIDAdam-AGRadGalNetdeep_sup-0.01
66	0402-MiraBestNOUNC-playgroundv2_deep_sup-0.001
67	0709Exp2-range_norm-MiraBestNOHYBRIDAdam-AGRadGalNetdeep_sup-5e-05
	Evaluating ...
PATH: TrainedNetworks/0709Exp2-range_norm-MiraBestNOHYBRIDAdam-AGRadGalNetdeep_sup-5e-05
PATH: TrainedNetworks/0709Exp2-range_norm-MiraBestNOHYBRIDAdam-AGRadGalNetdeep_sup-5e-05/81.pt


/usr/local/anaconda-python-3.6/lib/python3.6/site-packages/torch/nn/functional.py:2479: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


68	0611-MiraBestNOHYBRIDAdam-playgroundv3_ft-0.0001
69	1427-FRDEEPNOUNCAdam-transfer_original-0.0001
70	0709Exp2-sigmoid-MiraBestNOHYBRIDAdam-AGRadGalNetmean-5e-05
	Evaluating ...
PATH: TrainedNetworks/0709Exp2-sigmoid-MiraBestNOHYBRIDAdam-AGRadGalNetmean-5e-05
PATH: TrainedNetworks/0709Exp2-sigmoid-MiraBestNOHYBRIDAdam-AGRadGalNetmean-5e-05/98.pt


...

/usr/local/anaconda-python-3.6/lib/python3.6/site-packages/torch/nn/functional.py:2479: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


220	0719Exp33-range_norm-MiraBestNOHYBRIDAdam-AGRadGalNetft-5e-05
221	1427-FRDEEPNOUNCSGD-transfer_original-0.0001
222	0709Exp1-range_norm-FRDEEPAdam-AGRadGalNetft-5e-05
	Evaluating ...
PATH: TrainedNetworks/0709Exp1-range_norm-FRDEEPAdam-AGRadGalNetft-5e-05
PATH: TrainedNetworks/0709Exp1-range_norm-FRDEEPAdam-AGRadGalNetft-5e-05/81.pt


/usr/local/anaconda-python-3.6/lib/python3.6/site-packages/torch/nn/functional.py:2479: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


223	0614-MiraBestNOHYBRIDAdam-AGSononet-1e-08


"\ndataset_selection = 'automatic'\npickle_in = open(f'evaluated_models_{dataset_selection}.pickle','rb')\nevaluated_models = pickle.load(pickle_in)\npickle_in.close()\n#"

In [6]:
def remove_key(key):
    data_options = ['FRDEEP-F','MiraBest','MiraBestNOHYBRID','MiraBestNOUNC','automatic']
    for dataset in data_options:
        file = open(f'evaluated_models_{dataset}.pickle','rb')
        evaluated_models = pickle.load(file)
        file.close()
        try:
            print(evaluated_models[key])
            print(file)
            del evaluated_models[key]
            file2 = open(f'evaluated_models_{dataset}.pickle','wb')
            pickle.dump(evaluated_models,file2)
            file2.close()
        except:
            print(f'\t Already deleted {key}')

In [7]:
if 'df' in locals():
    del df
# The data in the dictionaries is in the format (form):
form = ['data_train','net_name','date','lr','epoch','auc','friP','friN','friiN','friiP','frirecall','friirecall','friprecision','friiprecision','frif1','friif1','fpr','tpr','thresholds']
### Add models trained on each dataset
for data in data_options[:-1]:
    # Load in dict:
    pickle_in = open(f'evaluated_models_{data}.pickle','rb')
    evaluated_models = pickle.load(pickle_in)
    pickle_in.close()

    # Load dict into dataframe
    df_temp = pd.DataFrame.from_dict(evaluated_models,orient='index',columns=form)
    # Make keys of dict into their own column
    df_temp['model'] = df_temp.index
    # Add data_evaluation column
    if data == 'MiraBest':
        df_temp = df_temp.assign(data_eval = f'{data}HYBRID')
    else:
        df_temp = df_temp.assign(data_eval = f'{data}')
    
    # Assign df or stack onto df
    if 'df' not in locals(): #Initialise
        df = df_temp
    elif 'df' in locals(): #Stack
        df = pd.concat([df,df_temp],axis=0)
    else:
        print('Something went wrong!')
    
# Reset index to numeric digits
df = df.reset_index(drop=True)
df[df.date!='TEST'].sort_values(by='date',ascending=False).head()

,data_train,net_name,date,lr,epoch,auc,friP,friN,friiN,friiP,...,friirecall,friprecision,friiprecision,frif1,friif1,fpr,tpr,thresholds,model,data_eval
192,FRDEEP-F,transfer_original,3427,NaN,6.0,0.832509,13019.0,4989.0,4621.0,14811.0,...,0.748030,0.722956,0.762196,0.730420,0.755047,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 5.0505050505050505e-05, 0.00333333333333...","[1.999593, 0.99959296, 0.999315, 0.9993061, 0....",3427-FRDEEPNOUNCSGD-transfer_original-0.0001,MiraBestHYBRID
65,FRDEEP-F,transfer_original,3427,NaN,6.0,0.987322,7227.0,108.0,693.0,9972.0,...,0.989286,0.985276,0.935021,0.947493,0.961388,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 9.92063492063492e-05, 0.0038690476190476...","[1.9999108, 0.99991083, 0.9998266, 0.9998253, ...",3427-FRDEEPNOUNCSGD-transfer_original-0.0001,FRDEEP-F
319,FRDEEP-F,transfer_original,3427,NaN,6.0,0.746389,18325.0,9811.0,8315.0,18629.0,...,0.655028,0.651301,0.691397,0.669089,0.672721,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 3.516174402250352e-05, 0.000703234880450...","[1.9999986, 0.99999857, 0.9995369, 0.9995352, ...",3427-FRDEEPNOUNCSGD-transfer_original-0.0001,MiraBestNOHYBRID
446,FRDEEP-F,transfer_original,3427,NaN,6.0,0.831619,13070.0,5075.0,4570.0,14725.0,...,0.743687,0.720309,0.763151,0.730474,0.753293,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 5.0505050505050505e-05, 0.00681818181818...","[1.9995937, 0.9995938, 0.9991347, 0.99913174, ...",3427-FRDEEPNOUNCSGD-transfer_original-0.0001,MiraBestNOUNC
297,FRDEEP-F,transfer_original,1427,NaN,2.0,0.835892,25503.0,16669.0,1137.0,11771.0,...,0.413889,0.604738,0.911915,0.741237,0.569362,"[0.0, 0.011486486486486487, 0.0115990990990990...","[0.0, 0.0006680731364275668, 0.000843881856540...","[2.0, 1.0, 0.9999999, 0.99999976, 0.99999964, ...",1427-FRDEEPNOUNCAdam-transfer_original-0.0001,MiraBestNOHYBRID


In [ ]:
# Add accuracy column
df['accuracy']=(df.friP+df.friiP)/(df.friP+df.friiP+df.friiN+df.friN)
#accuracy = (df.friP.to_numpy()+df.friiP.to_numpy())/(df.friP.to_numpy()+df.friiP.to_numpy()+df.friiN.to_numpy()+df.friN.to_numpy()) #NUMPY ACCURACY

In [ ]:
df = df.sort_values(by=['accuracy','auc'],ascending=False)
df[(df.data_train != 'FRDEEP-F') & (df.net_name != 'transfer_original')][['auc','accuracy','data_train','data_eval','net_name','model','date']].head(5)

In [ ]:
df[(df.date!='0429') & (df.date!='3427') & (df.data_train != 'FRDEEP-F')] \
    [['auc','accuracy','data_train','data_eval','net_name','model','date']].head(5)

In [ ]:
# Sort df by AUC and then by acc
df = df.sort_values(by=['auc','accuracy'],ascending=False)

In [ ]:
# Find all 'FRDEEP-F' models:
df[['auc','accuracy','data_train','data_eval','net_name','model','date']][df['data_train']=='FRDEEP-F'].head()

In [ ]:
# Find all non 'FRDEEP-F' models:
df[['auc','accuracy','data_train','data_eval','net_name','model','date','epoch']][df['data_train']!='FRDEEP-F'].head(20)

In [ ]:
# Change dataframe to desired columns:
d = df[['auc','accuracy','data_train','data_eval','net_name','model','date']]
d.head(20)

In [ ]:
# Five Best FRDEEP Trained Models:
best_frdeep = ['playground-0211-0.1_180Epochs.pt',
               'playground-0129-0.1_360Epochs.pt',
               'playground-0212-0.1_360Epochs.pt',
               'playground-0129-0.1_500Epochs.pt',
               '0224-playgroundv1']
d[d.model==best_frdeep[0]]

In [ ]:
d[d.model==best_frdeep[-1]]

In [ ]:
# Best MiraBest performer:
best_mirabest = ['0427-MiraBestNOHYBRIDAdam-playgroundv2_concat-0.0001',
                 '0310-MiraBestSGD-playgroundv2_ft',
                 '0309-MiraBest-playgroundv2_ft',
                 '0331-MiraBestNOHYBRID-playgroundv2_deep_sup',
                 '0310-MiraBestSGD-playgroundv2_ft',
                 '0305-MiraBest-playgroundv2_deep_sup']
d[d.model==best_mirabest[0]]

In [ ]:
d[d.model==best_mirabest[1]]

In [ ]:
best_models = best_frdeep+best_mirabest
avg_auc_max = 0 ; auc_model = ''
avg_acc_max = 0 ; acc_model = ''

for model in best_models:
    temp = d[d.model==model].mean(axis=0)
    if temp['auc']>avg_auc_max:
        avg_auc_max=temp['auc']
        auc_model=model
    if temp['accuracy']>avg_acc_max:
        avg_acc_max=temp['accuracy']
        acc_model=model

print(f"""
Best AUC avg: {avg_auc_max:.4f} from: {auc_model}
Best ACC avg: {avg_acc_max:.4f} from: {acc_model}
""")

In [ ]:
# Best overall performer (avg)
if auc_model == acc_model:
    best_model = auc_model
    print(f'Our overall best model is: {best_model}')
else:
    print(f'No single best model. \n\tAUC Best: {auc_model}\n\tACC Best:{acc_model}')

In [ ]:
d[d.model=='0408-MiraBestNOHYBRIDSGD-playgroundv2_ft-0.0001']

In [ ]:
df[['data_train','data_eval','auc','accuracy','friprecision','friiprecision','frif1','friif1']][df.model==best_mirabest[1]]

In [ ]:
print(best_frdeep[-1])
df[['data_train','data_eval','auc','accuracy','friprecision','friiprecision','frif1','friif1']][df.model==best_frdeep[-1]]